In [277]:
from uniswappy import *
import pandas as pd
from scipy import optimize
import copy
import matplotlib.pyplot as plt

user_nm = MockAddress().apply()
eth_amount = 10000
tkn_amount = 100000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [2]:
def calc_Lx(sqrtP, dx, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Lx = dx/(1/p_sqrt_human - 1/pb_sqrt_human)
    return Lx

def calc_Ly(sqrtP, dy, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Ly = dy/(p_sqrt_human - pa_sqrt_human)
    return Ly

def calc_y(lp, dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dy = dL*(p_sqrt_human - pa_sqrt_human)
    #y = dL*(pb_sqrt_human - pa_sqrt_human)
    return dy

def calc_x(lp, dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dx = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    #x = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    return dx

def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

In [287]:
def calc_lp_settlement(lp, token_in, itkn_amt, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    if(token_in.token_name == lp.token0):
        sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
        sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
        sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
        dPy = (sqrtp_cur - sqrtp_pa)
        dPx = (1/sqrtp_cur - 1/sqrtp_pb)          
    elif(token_in.token_name == lp.token1):
        sqrtp_cur = 2**96/lp.slot0.sqrtPriceX96
        sqrtp_pa = 2**96/TickMath.getSqrtRatioAtTick(lwr_tick)
        sqrtp_pb = 2**96/TickMath.getSqrtRatioAtTick(upr_tick)
        dPx = (1/sqrtp_cur - 1/sqrtp_pa)
        dPy = (sqrtp_cur - sqrtp_pb)

    fee = 997
    a = fee*dPy*sqrtp_cur*dPx - 1000*dPx*(sqrtp_cur**2) - fee*dPy  
    b = -fee*dPy*sqrtp_cur*itkn_amt + 1000*itkn_amt*(sqrtp_cur**2) + L*fee*dPy + 1000*L*dPx*(sqrtp_cur**2)
    c = -1000*L*itkn_amt*(sqrtp_cur**2)

    return (-b + math.sqrt(b*b - 4*a*c)) / (2*a)

def calc_tkn_settlement(lp, token_in, dL, lwr_tick, upr_tick):
            
    L = lp.get_liquidity()
    if(token_in.token_name == lp.token0):
        sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
        sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
        sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
        dPy = (sqrtp_cur - sqrtp_pa)
        dPx = (1/sqrtp_cur - 1/sqrtp_pb)          
    elif(token_in.token_name == lp.token1):
        sqrtp_cur = 2**96/lp.slot0.sqrtPriceX96
        sqrtp_pa = 2**96/TickMath.getSqrtRatioAtTick(lwr_tick)
        sqrtp_pb = 2**96/TickMath.getSqrtRatioAtTick(upr_tick)
        dPx = (1/sqrtp_cur - 1/sqrtp_pa)
        dPy = (sqrtp_cur - sqrtp_pb)

    dAmt = dL*dPx
    
    L_diff = (L - dL) 
    sqrtp_next = sqrtp_cur + (997*dy)/(L_diff*1000) 
    return dAmt + L_diff * (1/sqrtp_cur - 1/sqrtp_next)

def quote(lp, token_in, amt_tkn):
    
    fee = 997 
    L = lp.get_liquidity()
    if(token_in.token_name == lp.token0):
        sqrtp_cur = 2**96/lp.slot0.sqrtPriceX96
        sqrtp_pa = 2**96/TickMath.getSqrtRatioAtTick(lwr_tick)
        sqrtp_pb = 2**96/TickMath.getSqrtRatioAtTick(upr_tick)       
    elif(token_in.token_name == lp.token1):
        sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
        sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
        sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
 
    sqrtp_next = sqrtp_cur + (fee*amt_tkn) / (L*1000)
    return L * (1/sqrtp_cur - 1/sqrtp_next), sqrtp_next

def calc_deposit_portion(lp, token_in, amt_in):

    L = lp.get_liquidity()

    if(token_in.token_name == lp.token0):
        sqrtp_cur = 2**96/lp.slot0.sqrtPriceX96
        sqrtp_pa = 2**96/TickMath.getSqrtRatioAtTick(lwr_tick)
        sqrtp_pb = 2**96/TickMath.getSqrtRatioAtTick(upr_tick)  
        sqrtp_next = sqrtp_cur + (997*amt_in*0.5) / (L*1000)
    elif(token_in.token_name == lp.token1):
        sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
        sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
        sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96      
        sqrtp_next = sqrtp_cur + (997*amt_in*0.5) / (L*1000)
        
    dPy = (sqrtp_next - sqrtp_pa)
    dPx = (1/sqrtp_next - 1/sqrtp_pb)     

    A = L/(dPx*amt_in)
    B = 997*amt_in
    C = 1000*L
    D = 1000*L*(sqrtp_cur-sqrtp_pa)
    
    a = -A*B*B - B*C*sqrtp_cur
    b = -C*C*sqrtp_cur*sqrtp_cur - A*B*D + B*C*sqrtp_cur
    c = C*C*sqrtp_cur*sqrtp_cur
    
    alpha = (-b - math.sqrt(b*b - 4*a*c)) / (2*a)
    return alpha

In [520]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(10, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

tick_gap = 1000

lwr_tick = lp.get_tick_price(-1, 10, tick_gap)
upr_tick = lp.get_tick_price(1, 10, tick_gap)

sqrtP =  math.sqrt(10)
Ly = calc_Ly(sqrtP, 10000, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1000, lwr_tick, upr_tick)

lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Ly)    #3162.2776601683795
lp.summary()

lwr_tick = lp.get_tick_price(-1, lp.get_price(eth), tick_gap)
upr_tick = lp.get_tick_price(1, lp.get_price(eth), tick_gap)

amount_out = 1000
Lx = calc_Lx(sqrtP, amount_out, lwr_tick, upr_tick)
Ly = calc_Ly(sqrtP, amount_out, lwr_tick, upr_tick)
#p_out = calc_withdraw_portion(lp, tkn, amount_out, lwr_tick, upr_tick)

print(f'lwr_tick {lwr_tick}, cur_tick {price_to_tick(lp.get_price(eth))}, upr_tick {upr_tick}')

#dL_calc = calc_lp_settlement(lp, tkn, amount_out, lwr_tick, upr_tick)

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 967.0469739529013, TKN = 10000.0
Gross Liquidity: 64403.320664241655 

lwr_tick 22020, cur_tick 23027, upr_tick 24000


In [521]:
amt_eth1 = 100
alpha = opt_deposit_portion(lp, eth, amt_eth1, lwr_tick, upr_tick)

swap_in = amt_eth1*alpha
amt_tkn1, sqrtp_cur  = quote(lp, eth, swap_in)
sqrtp_cur = 1/sqrtp_cur
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96
dPy = (sqrtp_cur - sqrtp_pa)
dPx = (1/sqrtp_cur - 1/sqrtp_pb) 

dL = amt_tkn1/dPy

#Lx = dx/(1/p_sqrt_human - 1/pb_sqrt_human)
#Ly = dy/(p_sqrt_human - pa_sqrt_human)
 
dy = dL*dPy
dx = dL*dPx
print(f'{dx} ETH and {dy} TKN added')
swap_in + dx

51.48411018857578 ETH and 482.5573283632727 TKN added


99.9999999822102

In [522]:
amt_tkn1 = 1000
alpha = opt_deposit_portion(lp, tkn, amt_tkn1, lwr_tick, upr_tick)

swap_in = amt_tkn1*alpha
amt_tkn0, sqrtp_cur  = quote(lp, tkn, swap_in)

sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
dPy = (sqrtp_cur - sqrtp_pa)
dPx = (1/sqrtp_cur - 1/sqrtp_pb) 

dL = amt_tkn0/dPx
 
dy = dL*dPy
dx = dL*dPx
print(f'{dx} ETH and {dy} TKN added')

swap_in + dy

46.671161213457 ETH and 530.8088377136537 TKN added


999.9999999902011

In [517]:
def opt_deposit_portion(lp, tkn, amt_tkn_in, lwr_tick, upr_tick):
    bnds = [(0.35, 0.65)]
    opt_tol = 1e-8
    res = optimize.minimize(obj_func, x0 = 0.5, bounds=bnds, 
                            args=(amt_tkn_in, lp, tkn, lwr_tick, upr_tick), 
                            method='Nelder-Mead', tol=opt_tol)
    return res.x[0]

def obj_func(alpha, amt_tkn_in, lp, token_in, lwr_tick, upr_tick):
    opt_tol = 1e-8         
    swap_in = amt_tkn_in*alpha
    amt_tkn0, sqrtp_cur  = quote(lp, token_in, swap_in)
    sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96    
    
    if(token_in.token_name == lp.token0):
        sqrtp_cur = 1/sqrtp_cur
        
    dPy = (sqrtp_cur - sqrtp_pa)
    dPx = (1/sqrtp_cur - 1/sqrtp_pb) 
    
    if(token_in.token_name == lp.token0):
        dL = amt_tkn0/dPy 
        amt_deposit = dL*dPx
    elif(token_in.token_name == lp.token1): 
        dL = amt_tkn0/dPx 
        amt_deposit = dL*dPy
    
    diff = amt_tkn_in - (swap_in + amt_deposit) 
    
    
    return abs(diff)+opt_tol

In [6]:
amt_in = 999.9996370541235

In [151]:
(amt_in - dy)/amt_tkn1

0.469191

In [152]:
(amt_in - dL*dPy)/amt_tkn1

0.469191

In [153]:
(amt_in - dL*(sqrtp_cur - sqrtp_pa))/amt_tkn1

0.469191

In [154]:
swap_in = amt_tkn1*alpha

L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 

sqrtp_next = sqrtp_cur + (997*swap_in) / (L*1000)
(amt_in - dL*(sqrtp_next - sqrtp_pa))/amt_tkn1

0.469191

In [11]:
dy = 1000
swap_in = dy*alpha

L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
(dy - dL*(sqrtp_next - sqrtp_pa))/dy

0.46919136294587654

In [12]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

dL = L * (1/sqrtp_cur - 1/ (sqrtp_cur + (997*swap_in) / (L*1000)) )/dPx
dL

3265.8144966362984

In [13]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

dL = (L/dPx) * (1/sqrtp_cur - 1/ (sqrtp_cur + (997*swap_in) / (L*1000)) )
dL

3265.8144966362984

In [14]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*swap_in) / (L*1000)
amt_tkn0 = L * (1/sqrtp_cur - 1/sqrtp_next)
dL = amt_tkn0/dPx

(amt_in - dL*((sqrtp_cur + (997*amt_tkn1*alpha) / (L*1000)) - sqrtp_pa))/amt_tkn1

0.469191

In [15]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

dL = L * (1/sqrtp_cur - 1/ (sqrtp_cur + (997*swap_in) / (L*1000)) )/dPx

(amt_in - dL*((sqrtp_cur + (997*amt_tkn1*alpha) / (L*1000)) - sqrtp_pa))/amt_tkn1

0.469191

In [16]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

dL = (L/dPx) * (1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*amt_tkn1*alpha))

(amt_in - dL*((sqrtp_cur + (997*amt_tkn1*alpha) / (L*1000)) - sqrtp_pa))/amt_tkn1

0.4691910000000407

In [17]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

dL = (L/dPx) * (1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*amt_tkn1*alpha))

(amt_in - dL*((sqrtp_cur + (997*amt_tkn1*alpha) / (L*1000)) - sqrtp_pa))/amt_tkn1

0.4691910000000407

In [18]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

dL = (L/dPx) * (1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*amt_tkn1*alpha))

(amt_in - dL*(  (sqrtp_cur + (997*amt_tkn1*alpha) / (L*1000)) - sqrtp_pa))/amt_tkn1

0.4691910000000407

In [19]:
dy = 1000
alpha = 0.4691913629459173

L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

dL = (L/dPx) * (1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*dy*alpha))

(dy - dL*(  (sqrtp_cur + (997*dy*alpha) / (L*1000)) - sqrtp_pa))/dy

0.4691909141529487

In [20]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

(dy - (L/dPx) * (1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*dy*alpha))*(  (sqrtp_cur + (997*dy*alpha) / (L*1000)) - sqrtp_pa))/dy 

0.4691909141529487

In [21]:
L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96

sqrtp_next = sqrtp_cur + (997*dy*alpha) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

1 - (L/(dPx*dy)) * (1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*dy*alpha))*(  (sqrtp_cur + (997*dy*alpha) / (L*1000)) - sqrtp_pa)

0.46919091415294867

In [22]:
1 - (L/(dPx*dy)) * (  1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*dy*alpha)  )*(  (1000*L*sqrtp_cur + 997*dy*alpha - 1000*L*sqrtp_pa)/(1000*L) ) 

0.4691909141529491

In [23]:
1/sqrtp_cur - L*1000/ (L*1000*sqrtp_cur + 997*dy*alpha) 

0.0007246704152402139

In [24]:
(L*1000*sqrtp_cur + 997*dy*alpha - L*1000*sqrtp_cur)  / (sqrtp_cur*(L*1000*sqrtp_cur + 997*dy*alpha)) 

0.0007246704152401749

In [25]:
1 - (L/(dPx*dy)) * (  (L*1000*sqrtp_cur + 997*dy*alpha - L*1000*sqrtp_cur)  / (L*1000*sqrtp_cur**2 + 997*dy*alpha*sqrtp_cur)  )*(  (1000*L*sqrtp_cur + 997*dy*alpha - 1000*L*sqrtp_pa)/(1000*L) ) 

0.46919091415297787

In [26]:
1 - (L/(dPx*dy)) * (  (997*dy*alpha)  / (L*1000*sqrtp_cur**2 + 997*dy*alpha*sqrtp_cur)  )*(  (1000*L*sqrtp_cur + 997*dy*alpha - 1000*L*sqrtp_pa)/(1000*L) ) 

0.46919091415297

In [27]:
1 - (L/(dPx*dy)) * (  (997*dy*alpha)  / (L*1000*sqrtp_cur**2 + 997*dy*alpha*sqrtp_cur)  )*(  (1000*L*(sqrtp_cur-sqrtp_pa) + 997*dy*alpha)/(1000*L) ) 

0.46919091415296965

In [28]:
A = L/(dPx*dy)
B = 997*dy
C = 1000*L
D = 1000*L*(sqrtp_cur-sqrtp_pa)

1 - A * (  (B*alpha)  / (C*sqrtp_cur**2 + B*alpha*sqrtp_cur)  )*(  (D + B*alpha)/(C) ) 

0.46919091415296965

In [29]:
1 - (A*B*alpha)*(D+B*alpha)/(C*C*sqrtp_cur**2 + C*B*alpha*sqrtp_cur)

0.46919091415296965

In [30]:
ls = 1 - (A*B*alpha)*(D+B*alpha)/(C*C*sqrtp_cur**2 + C*B*alpha*sqrtp_cur)
ls = 0.4691909349268425

In [31]:
1 - ls - (A*B*alpha)*(D+B*alpha)/(C*C*sqrtp_cur**2 + C*B*alpha*sqrtp_cur)

-2.077387284415977e-08

In [32]:
(C*C*sqrtp_cur**2 + C*B*alpha*sqrtp_cur) - ls*(C*C*sqrtp_cur**2 + C*B*alpha*sqrtp_cur) - (A*B*alpha)*(D+B*alpha)

-863635260.0

In [33]:
-C*C*(ls - 1)*sqrtp_cur*sqrtp_cur - B*alpha*C*(ls - 1)*sqrtp_cur - A*B*alpha*(B*alpha + D)

-863635260.0

In [34]:
(-A*B*B - B*C*sqrtp_cur)*alpha*alpha + (-C*C*sqrtp_cur*sqrtp_cur - A*B*D + B*C*sqrtp_cur)*alpha + C*C*sqrtp_cur*sqrtp_cur

-18657734984.0

In [35]:
ls*(C*C*sqrtp_cur**2) + ls*C*B*alpha*sqrtp_cur  - C*C*sqrtp_cur**2 - C*B*alpha*sqrtp_cur + (A*B*alpha)*(D+B*alpha)

863635260.0

In [36]:
ls*(C*C*sqrtp_cur**2) + ls*C*B*alpha*sqrtp_cur  - C*C*sqrtp_cur**2 - C*B*alpha*sqrtp_cur + A*B*D*alpha + (A*B*B)*alpha**2

863635261.0

In [37]:
ls*(C*C*sqrtp_cur**2) + ls*C*B*alpha*sqrtp_cur  - C*C*sqrtp_cur**2 - C*B*alpha*sqrtp_cur + A*B*D*alpha + (A*B*B)*alpha**2

863635261.0

In [38]:
ls*(C*C*sqrtp_cur**2) + ls*C*B*alpha*sqrtp_cur  - C*C*sqrtp_cur**2 - C*B*alpha*sqrtp_cur + A*B*D*alpha + (A*B*B)*alpha**2

863635261.0

In [39]:
(A*B*B)*alpha**2 + (A*B*D + C*C*sqrtp_cur**2)*alpha - C*C*sqrtp_cur**2 

50588477805712.0

In [40]:
dy = 1000
L = lp.get_liquidity()

sqrtp_cur = lp.slot0.sqrtPriceX96/2**96

dPy = (sqrtp_cur - sqrtp_pa)
dPx = (1/sqrtp_cur - 1/sqrtp_pb) 

A = L/(dPx*dy)
B = 997*dy
C = 1000*L
D = 1000*L*(sqrtp_cur-sqrtp_pa)

a = -A*B*B - B*C*sqrtp_cur
b = -C*C*sqrtp_cur*sqrtp_cur - A*B*D + B*C*sqrtp_cur
c = C*C*sqrtp_cur*sqrtp_cur

p_in = (-b - math.sqrt(b*b - 4*a*c)) / (2*a)
p_in

0.4812547663961031

In [42]:
dy = 1000

L = lp.get_liquidity()
sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 

sqrtp_next = sqrtp_cur + (997*dy*0.5) / (L*1000)
dPy = (sqrtp_next - sqrtp_pa)
dPx = (1/sqrtp_next - 1/sqrtp_pb) 

A = L/(dPx*dy)
B = 997*dy
C = 1000*L
D = 1000*L*(sqrtp_cur-sqrtp_pa)

a = -A*B*B - B*C*sqrtp_cur
b = -C*C*sqrtp_cur*sqrtp_cur - A*B*D + B*C*sqrtp_cur
c = C*C*sqrtp_cur*sqrtp_cur

p_in = (-b - math.sqrt(b*b - 4*a*c)) / (2*a)
p_in

0.4683808565248589

In [ ]:
amt_tkn1 = 1000
#p_in = 0.469191

swap_in = amt_tkn1*p_in
amt_tkn0, sqrtp_cur  = quote(lp, tkn, swap_in)

sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
dPy = (sqrtp_cur - sqrtp_pa)
dPx = (1/sqrtp_cur - 1/sqrtp_pb) 

dL = amt_tkn0/dPx
 
dy = dL*dPy
dx = dL*dPx
print(f'{dx} ETH and {dy} TKN added')

swap_in + dy

In [74]:
calc_deposit_portion(lp, tkn, 1000)

0.44740139848813903

In [527]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(10, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

tick_gap = 1000

lwr_tick = lp.get_tick_price(-1, 10, tick_gap)
upr_tick = lp.get_tick_price(1, 10, tick_gap)

sqrtP =  math.sqrt(10)
Ly = calc_Ly(sqrtP, 10000, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1000, lwr_tick, upr_tick)

lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Ly)    #3162.2776601683795
lp.summary()

lwr_tick = lp.get_tick_price(-1, lp.get_price(eth), tick_gap)
upr_tick = lp.get_tick_price(1, lp.get_price(eth), tick_gap)

amount_out = 1000
Lx = calc_Lx(sqrtP, amount_out, lwr_tick, upr_tick)
Ly = calc_Ly(sqrtP, amount_out, lwr_tick, upr_tick)
#p_out = calc_withdraw_portion(lp, tkn, amount_out, lwr_tick, upr_tick)

print(f'lwr_tick {lwr_tick}, cur_tick {price_to_tick(lp.get_price(eth))}, upr_tick {upr_tick}')

# Calculate LP settlement
#alpha = calc_deposit_portion(lp, tkn, amount_out)
alpha = opt_deposit_portion(lp, tkn, amount_out, lwr_tick, upr_tick)

# Swap
out = Swap().apply(lp, tkn, user_nm, alpha*amount_out)
lp.summary()

print(f'---- Swap {amt1:.6} TKN for {abs(out[1]):.6} ETH ----\n')

sqrtP = lp.slot0.sqrtPriceX96/2**96
dL_calc = calc_Lx(sqrtP, abs(out[1]), lwr_tick, upr_tick)

# Deposit
out = lp.mint(user_nm, lwr_tick, upr_tick, dL_calc)
lp.summary()

print(f'---- Withdraw {dL_calc:.6} LP tokens for {amt0:.6} ETH and {amt1:.6} TKN ----\n')
out

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 967.0469739529013, TKN = 10000.0
Gross Liquidity: 64403.320664241655 

lwr_tick 22020, cur_tick 23027, upr_tick 24000
Exchange ETH-TKN (LP)
Real Reserves:   ETH = 920.3758127394445, TKN = 10469.191162276547
Gross Liquidity: 64403.320664241655 

---- Swap 5470.56 TKN for 46.6712 ETH ----

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 967.0469739529013, TKN = 10999.9999999902
Gross Liquidity: 67669.13634496479 

---- Withdraw 3265.82 LP tokens for 481.076 ETH and 5470.56 TKN ----



(46.671161213456784, 530.8088377136517)

In [526]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(10, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

tick_gap = 1000

lwr_tick = lp.get_tick_price(-1, 10, tick_gap)
upr_tick = lp.get_tick_price(1, 10, tick_gap)

sqrtP =  math.sqrt(10)
Ly = calc_Ly(sqrtP, 10000, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1000, lwr_tick, upr_tick)

lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Lx)    #3162.2776601683795
lp.summary()

lwr_tick = lp.get_tick_price(-1, lp.get_price(eth), tick_gap)
upr_tick = lp.get_tick_price(1, lp.get_price(eth), tick_gap)

amount_out = 100
Lx = calc_Lx(sqrtP, amount_out, lwr_tick, upr_tick)
Ly = calc_Ly(sqrtP, amount_out, lwr_tick, upr_tick)
#p_out = calc_withdraw_portion(lp, tkn, amount_out, lwr_tick, upr_tick)

print(f'lwr_tick {lwr_tick}, cur_tick {price_to_tick(lp.get_price(eth))}, upr_tick {upr_tick}')

# Calculate LP settlement
#alpha = calc_deposit_portion(lp, eth, amount_out)
alpha = opt_deposit_portion(lp, eth, amount_out, lwr_tick, upr_tick)

# Swap
out_swap = Swap().apply(lp, eth, user_nm, alpha*amount_out)
lp.summary()

print(f'---- Swap {alpha*amount_out:.6} ETH for {abs(out_swap[2]):.6} TKN ----\n')

sqrtP = lp.slot0.sqrtPriceX96/2**96
dL_calc = calc_Ly(sqrtP, abs(out_swap[2]), lwr_tick, upr_tick)

# Deposit
out = lp.mint(user_nm, lwr_tick, upr_tick, dL_calc)
lp.summary()

print(f'---- Deposit {dL_calc:.6} LP tokens for {amt0:.6} ETH and {amt1:.6} TKN ----\n')

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 999.9999999999997, TKN = 10340.759310919504
Gross Liquidity: 66597.92378128914 

lwr_tick 22020, cur_tick 23027, upr_tick 24000
Exchange ETH-TKN (LP)
Real Reserves:   ETH = 1048.5911921318616, TKN = 9857.41697704624
Gross Liquidity: 66597.92378128914 

---- Swap 48.5912 ETH for 483.342 TKN ----

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 1100.00000000956, TKN = 10340.759310919504
Gross Liquidity: 69863.44414899987 

---- Deposit 3265.52 LP tokens for 481.076 ETH and 5470.56 TKN ----



In [96]:
def calc_deposit_portion(lp, token_in, amt_in):

    L = lp.get_liquidity()

    if(token_in.token_name == lp.token0):
        sqrtp_cur = 2**96/lp.slot0.sqrtPriceX96
        sqrtp_pa = 2**96/TickMath.getSqrtRatioAtTick(lwr_tick)
        sqrtp_pb = 2**96/TickMath.getSqrtRatioAtTick(upr_tick)  
        sqrtp_next = sqrtp_cur + (997*amt_in*0.5) / (L*1000)
        dPy = (sqrtp_next - sqrtp_pa)
        dPx = (1/sqrtp_next - 1/sqrtp_pb)  
    
    elif(token_in.token_name == lp.token1):
        sqrtp_cur = lp.slot0.sqrtPriceX96/2**96
        sqrtp_pa = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
        sqrtp_pb = TickMath.getSqrtRatioAtTick(upr_tick)/2**96      
        sqrtp_next = sqrtp_cur + (997*amt_in*0.5) / (L*1000)
        dPy = (sqrtp_next - sqrtp_pa)
        dPx = (1/sqrtp_next - 1/sqrtp_pb)   

    A = L/(dPx*amt_in)
    B = 997*amt_in
    C = 1000*L
    D = 1000*L*(sqrtp_cur-sqrtp_pa)
    
    a = -A*B*B - B*C*sqrtp_cur
    b = -C*C*sqrtp_cur*sqrtp_cur - A*B*D + B*C*sqrtp_cur
    c = C*C*sqrtp_cur*sqrtp_cur
    
    alpha = (-b - math.sqrt(b*b - 4*a*c)) / (2*a)
    return alpha

In [ ]:
def calc_deposit_portion(lp, token_in, dx):

    tkn_supply = get_tkn_supply(lp, token_in)
    a = 997*(dx**2)/(1000*tkn_supply)
    b = dx*(1997/1000)
    c = -dx

    alpha = -(b - math.sqrt(b*b - 4*a*c)) / (2*a)
    return alpha 

def get_tkn_supply(lp, token_in):
    tokens = lp.factory.token_from_exchange[lp.name]
    if(token_in.token_name == lp.token0):
        tkn_supply = lp.get_reserve(tokens[lp.token0])
    else:    
        tkn_supply = lp.get_reserve(tokens[lp.token1])
    return tkn_supply